In [17]:
import networkx as nx
import os
import pandas as pd
import math
import constants as const

In [31]:
def get_coordinate_files(muestra, path_micelio):
    files_micelios = [path_micelio + str(muestra+1) + "/" + f for f in os.listdir(path_micelio+str(muestra+1)) if '.csv' in f]
    files_micelios.sort()
    return files_micelios

In [19]:
def get_nodes(ultima_muestra):
    df = pd.read_csv(ultima_muestra, names=["x1", "y1", "x2", "y2"], header=None)
     #Nodos 1
    x1 = df["x1"].tolist()
    y1 = df["y1"].tolist()
    nodes1 = [(f, b) for f, b in zip(x1, y1)]
    #Nodos 2
    x2 = df["x2"].tolist()
    y2 = df["y2"].tolist()
    nodes2 = [(f, b) for f, b in zip(x2, y2)]
    nodes = set()
    nodes.update(nodes1, nodes2)
    nodes = list(nodes)
    nodes.sort()
    nodes_dic = {nodes[i]:i for i in range(len(nodes))}
    return nodes_dic

In [20]:
def update_coordiante_file(nodes_dic, file_coordinates):
    df = pd.read_csv(file_coordinates, names=["x1", "y1", "x2", "y2"], header=None)
    nodes1, nodes2 = read_coordinates(file_coordinates)
    #Id nodos
    v1 = [nodes_dic[v] for v in nodes1]
    v2 = [nodes_dic[v] for v in nodes2]
    df["v1"] = v1
    df["v2"] = v2
    print(df)

In [21]:
def read_coordinates(file_coordinates):
    df = pd.read_csv(file_coordinates, names=["x1", "y1", "x2", "y2"], header=None)
    #Nodos 1
    x1 = df["x1"].tolist()
    y1 = df["y1"].tolist()
    nodes1 = [(f, b) for f, b in zip(x1, y1)]
    #Nodos 2
    x2 = df["x2"].tolist()
    y2 = df["y2"].tolist()
    nodes2 = [(f, b) for f, b in zip(x2, y2)]
    return nodes1, nodes2

In [37]:
def adjust_coordinates(nodes_dic, file_coordinates):
    df = pd.read_csv(file_coordinates, names=["x1", "y1", "x2", "y2"], header=None)
    nodes1, nodes2 = read_coordinates(file_coordinates)
    #Id nodos
    v1=list()
    v2=list()
    for v in nodes1:
        dist = [math.dist(v, v_ref) for v_ref in nodes_dic.keys()]
        v_adj = dist.index(min(dist))
        if min(dist) != 0:
            print(min(dist))
        #print(min(dist), " node ", str(v_adj)) 
        v1.append(v_adj)
    for v in nodes2:
        dist = [math.dist(v, v_ref) for v_ref in nodes_dic.keys()]
        #print("Distancias")
        v_adj = dist.index(min(dist))
        #print(min(dist), " node ", str(v_adj)) 
        v2.append(v_adj)
    df["v1"] = v1
    df["v2"] = v2
    print(df)

In [51]:
path_micelio = const.PATH_NEUROSPORA
for m in range(1): 
    # 1.- Obtiene las rutas de las muestras
    f_micelios = get_coordinate_files(m, path_micelio)

    # 2.- Obtiene un diccionario coordenada:nodo, para la ultima muestra
    ultima_muestra = f_micelios[-2]
    nodes_dic = get_nodes(ultima_muestra)

    # 3.- Actualiza el archivo de coordenadas, asignando un id a cada nodo
    update_coordiante_file(nodes_dic, ultima_muestra)
    print("Muestra ", str(m+1))
    for i in range(len(f_micelios)):
        #print(f_micelios[i])
        #4.- Ajusta las coordenadas de acuerdo con el diccionario coordenada:nodo 
        adjust_coordinates(nodes_dic, f_micelios[i])


         x1      y1      x2      y2   v1   v2
0   -2261.5  1560.0 -2317.5  1525.0   70   67
1    3571.0 -1676.5  3465.0 -1744.0  173  167
2    3403.0  -330.5  3552.5  -264.5  162  172
3    3403.0  -330.5  2955.0  -502.0  162  156
4    3403.0  -330.5  3622.5  -499.5  162  177
..      ...     ...     ...     ...  ...  ...
220 -5004.0  1387.5 -4733.0  1144.5   13   19
221    95.5  -213.5    74.5  -153.5  105  102
222  4710.5 -1416.0  4618.5 -1466.5  199  198
223  4618.5 -1466.5  4405.0 -1095.0  198  195
224 -2532.5   148.0 -2172.5   -18.0   59   72

[225 rows x 6 columns]
Muestra  1
3.5355339059327378
3.5355339059327378
3.5355339059327378
18.741664813991314
8.514693182963201
94.24038412485382
5.1478150704935
5.1478150704935
131.18307817702708
14.916433890176299
32.59601202601324
4.242640687119285
4.242640687119285
3.5355339059327378
3.5355339059327378
3.5355339059327378
1.8027756377319946
39.5790348543266
39.5790348543266
3.5355339059327378
3.5355339059327378
2.0615528128088303
2.69258240

In [49]:

net = "redes/Neurospora_crassa/"
#probar si son subgrafos
files_net = [net + f for f in os.listdir(net) if '.gexf' in f]
files_net.sort()
nets = [nx.read_gexf(f) for f in files_net]
for i in range(len(nets)-1):
    G1 = nx.read_gexf(files_net[i])
    G2 = nx.read_gexf(files_net[i+1])
    print("Graph 1", files_net[i], " es árbol ", nx.is_tree(G1))
    print("Graph 2", files_net[i+1], " es árbol ", nx.is_tree(G2))
    isomatcher = nx.isomorphism.GraphMatcher(G2, G1)
    print("Son isomorfos", isomatcher.subgraph_is_isomorphic())

Graph 1 redes/Neurospora_crassa/Muestra1_M1N17h.gexf  es árbol  True
Graph 2 redes/Neurospora_crassa/Muestra1_M1N18h.gexf  es árbol  True
Son isomorfos True
Graph 1 redes/Neurospora_crassa/Muestra1_M1N18h.gexf  es árbol  True
Graph 2 redes/Neurospora_crassa/Muestra1_M1N20h.gexf  es árbol  True
Son isomorfos True
Graph 1 redes/Neurospora_crassa/Muestra1_M1N20h.gexf  es árbol  True
Graph 2 redes/Neurospora_crassa/Muestra1_M1N22h.gexf  es árbol  True
Son isomorfos True
Graph 1 redes/Neurospora_crassa/Muestra1_M1N22h.gexf  es árbol  True
Graph 2 redes/Neurospora_crassa/Muestra1_M1N24h.gexf  es árbol  False
Son isomorfos False
Graph 1 redes/Neurospora_crassa/Muestra1_M1N24h.gexf  es árbol  False
Graph 2 redes/Neurospora_crassa/Muestra2_M2N18h.gexf  es árbol  True
Son isomorfos False
Graph 1 redes/Neurospora_crassa/Muestra2_M2N18h.gexf  es árbol  True
Graph 2 redes/Neurospora_crassa/Muestra2_M2N20h.gexf  es árbol  True
Son isomorfos True
Graph 1 redes/Neurospora_crassa/Muestra2_M2N20h.gexf  

Traceback (most recent call last):
  File "/Users/daniela/anaconda3/envs/networks/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/gw/x5b6glr13clbt8md6003thh40000gn/T/ipykernel_9848/1398291084.py", line 12, in <module>
    print("Son isomorfos", isomatcher.subgraph_is_isomorphic())
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daniela/anaconda3/envs/networks/lib/python3.11/site-packages/networkx/algorithms/isomorphism/isomorphvf2.py", line 364, in subgraph_is_isomorphic
    x = next(self.subgraph_isomorphisms_iter())
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daniela/anaconda3/envs/networks/lib/python3.11/site-packages/networkx/algorithms/isomorphism/isomorphvf2.py", line 384, in subgraph_isomorphisms_iter
    yield from self.match()
  File "/Users/daniela/anaconda3/envs/networks/lib/python3.11/site-packages/networkx/algori